In [63]:
from gym import Env
import gym
from gym.spaces import Discrete, Box
import numpy as np
import random

In [64]:
class RobotEnvironment(Env):
    def __init__(self, angle_increment = 0.5):
        self.angle_increment = angle_increment
        self.action_space = Discrete(3) # this will chenge the joint actions by maintaining, increasing or decreasing the angle. multiply angle static increment by sample action
        self.observation_space = Box(low=np.array([-90]), high=np.array([90])) # why is this a numpy array?
        self.state = 0
        self.episode_lenght = 60
        
    def step(self, action):
        self.state += ((action-1)*self.angle_increment)
        self.episode_lenght -= 1
    
        if self.state<10 and self.state>1:
            reward = 1
        else:
            reward = -1
        
        done = False

        if self.episode_lenght<=0:
            done = True
        self.state += random.randint(-1,1)
        info = {}

        return self.state, reward, done, info

    def render(self, mode):
        mode = mode
        print(self.state)

    def reset(self):
        self.state = 0
        self.episode_lenght = 60
        return self.state

In [65]:
env = RobotEnvironment()
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [66]:
states = env.observation_space.shape
actions = env.action_space.n

In [67]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [68]:
model = build_model(states, actions)

In [69]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 24)                48        
                                                                 
 dense_16 (Dense)            (None, 24)                600       
                                                                 
 dense_17 (Dense)            (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [70]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [71]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [72]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=20000, visualize=False, verbose=1)

2021-12-11 18:34:02.315218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:02.365737: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:02.417345: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:02.469494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Training for 20000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 22:29 - reward: 1.0000

2021-12-11 18:34:02.564957: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:02.615959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:02.629011: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:02.717341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:02.837562: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:02.893271: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:02.911893: I tensorflow/core/grappler/optimizers/cust

   12/10000 [..............................] - ETA: 10:40 - reward: 1.0000

2021-12-11 18:34:03.179816: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:03.240937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:03.297053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-11 18:34:03.351304: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10000/10000 [==============================] - 123s 12ms/step - reward: 0.6786
166 episodes - episode_reward: 40.639 [-36.000, 60.000] - loss: 3.882 - mae: 13.511 - mean_q: 17.549

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 122s 12ms/step - reward: 0.9978
done, took 244.694 seconds


In [73]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 60.000, st

In [74]:
dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
-1
0
-1
-1
0
1
0
1
1
1
0
1
0
1
0
-1
0
-1
0
0
-1
-2
-1
-1
-1
0
1
1
1
0
1
2
1
2
3
3
2
1
1
0
1
0
0
1
2
1
1
1
0
-1
-2
-1
0
0
1
0
-1
-1
-1
-2
Episode 1: reward: 60.000, steps: 60
-1
0
1
2
1
1
2
3
3
4
3
3
2
1
0
1
0
1
2
2
3
4
3
3
4
5
5
6
7
8
7
8
8
7
6
7
7
7
8
9
8
7
6
6
7
6
5
5
6
7
7
7
8
9
8
8
8
9
8
8
Episode 2: reward: 60.000, steps: 60
-1
0
0
0
0
1
1
2
3
2
1
2
1
0
0
-1
0
-1
0
-1
-2
-3
-3
-4
-3
-2
-2
-1
-2
-2
-2
-2
-1
-2
-1
0
0
-1
0
0
-1
-1
0
0
-1
-1
-2
-2
-2
-3
-4
-5
-3
-2
-2
-2
-2
-2
-1
0
Episode 3: reward: 60.000, steps: 60
-1
0
1
0
0
-1
-1
0
0
0
0
-1
-2
-3
-3
-4
-3
-2
-2
-2
-3
-2
-1
0
1
0
1
0
1
1
0
0
1
1
0
-1
-2
-1
0
0
1
1
1
1
0
1
1
1
2
3
2
3
4
3
3
3
4
3
4
5
Episode 4: reward: 60.000, steps: 60
0
0
-1
0
0
-1
-2
-1
0
-1
-2
-1
0
1
1
2
1
0
0
0
-1
-1
0
-1
0
-1
-2
-1
-1
-2
-2
-1
-1
-1
-2
-1
-2
-2
-1
0
0
-1
-1
0
0
0
-1
-2
-2
-3
-3
-3
-2
-3
-3
-3
-4
-3
-2
-3
Episode 5: reward: 60.000, steps: 60
-1
0
1
0
-1
-1
-1
0
-1
-2
-1
-2
-1
-1
-2
-2
-3
-3
-2
-2
-3
-3
-3
-2
-2
-1
